**Read .ply file of point cloud**

In [1]:
import open3d as o3d
import numpy as np
import os
import h5py

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
script_dir = os.path.dirname(os.getcwd())
sim_pc_path = os.path.join(script_dir, '.', 'docs', 'smartLab_simulated.ply')

pcd = o3d.io.read_point_cloud(sim_pc_path)
points = np.asarray(pcd.points)          # Shape: [N, 3]
colors = np.asarray(pcd.colors)           # Shape: [N, 3]

#o3d.visualization.draw_geometries([pcd])

Features of dataset (9):
coordinates(3), colors(3), normalized coordinates(3)

In [3]:
def normalize_points(points):
    # Step 1: Center the points around the origin
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid

    # Step 2: Scale to fit within the [0, 1] interval
    min_vals = np.min(centered_points, axis=0)
    max_vals = np.max(centered_points, axis=0)
    scale = max_vals - min_vals

    # Avoid division by zero in case of flat dimension
    scale[scale == 0] = 1  # Set zero scales to 1 to keep that dimension as 0.5 after normalization

    normalized_points = (centered_points - min_vals) / scale

    return normalized_points

normalized = normalize_points(np.array(pcd.points))

In [4]:
# Concatenate to get features of shape (N, 9)
features = np.concatenate([points, colors, normalized], axis=1)

In [5]:
color_map = np.array([
    [1.0, 0.0, 0.0],  # Label 0: Red,  'ceiling'
    [0.0, 1.0, 0.0],  # Label 1: Green, 'floor'
    [0.0, 0.0, 1.0],  # Label 2: Blue,  'wall'
    [1.0, 1.0, 0.0],  # Label 3: Yellow, 'beam'
    [1.0, 0.0, 1.0],  # Label 4: Magenta, 'column'
    [0.0, 1.0, 1.0],  # Label 5: Cyan, 'window'
    [0.5, 0.5, 0.5],  # Label 6: Gray, 'door'
    [1.0, 0.5, 0.0],  # Label 7: Orange, 'chair'
    [0.5, 0.0, 1.0],  # Label 8: Purple, 'table'
    [0.5, 1.0, 0.5],  # Label 9: Light Green, 'bookcase'
    [0.5, 0.5, 1.0],  # Label 10: Light Blue, 'sofa'
    [1.0, 0.5, 0.5],  # Label 11: Pink, 'board'
    [0.0, 0.0, 0.0]   # Label 12: Black, 'clutter'
    ])

# Find closest color in color_map for each point
#labels = np.argmin(np.linalg.norm(colors[:, None] - color_map, axis=2), axis=1)

**Blocks of 1m x 1m x 1m with 4096 points in each like S3DIS dataset**

In [6]:
min_bound = np.min(points, axis=0)
max_bound = np.max(points, axis=0)

block_size = 1.0  # in meters
num_blocks_x = int(np.ceil((max_bound[0] - min_bound[0]) / block_size))
num_blocks_y = int(np.ceil((max_bound[1] - min_bound[1]) / block_size))
num_blocks_z = int(np.ceil((max_bound[2] - min_bound[2]) / block_size))

block_features_list = []
block_labels_list  = []

for ix in range(num_blocks_x):
    for iy in range(num_blocks_y):
        for iz in range(num_blocks_z):
            # Define the spatial boundaries for this block
            x_min = min_bound[0] + ix * block_size
            x_max = x_min + block_size
            y_min = min_bound[1] + iy * block_size
            y_max = y_min + block_size
            z_min = min_bound[2] + iz * block_size
            z_max = z_min + block_size

            # Find indices of points within the block
            in_block = np.where(
                (points[:, 0] >= x_min) & (points[:, 0] < x_max) &
                (points[:, 1] >= y_min) & (points[:, 1] < y_max) &
                (points[:, 2] >= z_min) & (points[:, 2] < z_max)
            )[0]

            if len(in_block) == 0:
                continue  # Skip empty blocks

            block_features = features[in_block, :]
            # If label is directly extractable, substitute here. If not, infer from color.
            block_colors = colors[in_block]
            block_labels = np.argmin(np.linalg.norm(colors[in_block][:, None] - color_map, axis=2), axis=1)

            # --- Handling Block Size (4096 points) ---
            # If there are more than 4096 points, randomly sample 4096.
            # If there are fewer, perform random duplication (or padding with zeros) to reach 4096.
            num_points = block_features.shape[0]
            target_points = 4096

            if num_points >= target_points:
                idx = np.random.choice(num_points, target_points, replace=False)
            else:
                # Duplicate some points
                idx = np.concatenate([
                    np.arange(num_points),
                    np.random.choice(num_points, target_points - num_points, replace=True)
                ])
            block_features = block_features[idx, :]
            block_labels = block_labels[idx]

            block_features_list.append(block_features)
            block_labels_list.append(block_labels)

In [7]:
data_array = np.stack(block_features_list, axis=0)  # Shape: (B, 4096, 9)
label_array = np.stack(block_labels_list, axis=0)     # Shape: (B, 4096)

print(data_array[0][0])
print(label_array[0][0])



[-5.6099239  -5.96243998  0.34489385  0.          0.          0.
  0.01828092  0.0707432   0.17926242]
12


In [8]:
sim_pc_data_path = os.path.join(script_dir, '.', 'docs', 'sim_pc_dataset.h5')

# Save to HDF5
with h5py.File(sim_pc_data_path, 'w') as f:
    f.create_dataset('data', data=data_array, compression='gzip')
    f.create_dataset('label', data=label_array, compression='gzip')

In [9]:
import h5py
f = h5py.File(sim_pc_data_path, 'r') #train_pc_path, data_file_path
list(f.keys())
dset = f['data']
dset.shape

(485, 4096, 9)

In [10]:
from pprint import pprint
pprint(dset[0][2].tolist())

[-5.620427605685707,
 -6.410465555278665,
 -0.0744237747039048,
 0.0,
 1.0,
 0.0,
 0.017368742185738886,
 0.030458044305317365,
 0.07192523227401235]


In [11]:
dlabel = f['label']
dlabel.shape

(485, 4096)